<a href="https://colab.research.google.com/github/yoshi4869/DriveRecTest/blob/main/DLTcoaching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import zipfile
import re
import os
import sys

# DLTログの行を表現するためのクラス
class DltLogEntry:
    def __init__(self, timestamp, level, apid, ctid, message, original_line):
        self.timestamp = timestamp
        self.level = level
        self.apid = apid
        self.ctid = ctid
        self.message = message
        self.original_line = original_line

    def __str__(self):
        # ログレベルに応じて色を付ける（ターミナルがANSIエスケープシーケンスをサポートしている場合）
        color_map = {
            'FAT': '\033[91m',  # 赤
            'ERR': '\033[31m',  # 明るい赤
            'WRN': '\033[33m',  # 黄色
            'INF': '\033[92m',  # 緑
            'DEB': '\033[94m',  # 青
            'VER': '\033[90m',  # 濃い灰色 (VERBOSE)
            'RES': '\033[0m'    # リセット
        }

        # 時系列順に読む練習のため、行番号を付与
        level_color = color_map.get(self.level, color_map['RES'])
        return f"[{self.timestamp}] {level_color}{self.level}{color_map['RES']} {self.apid}/{self.ctid}: {self.message}"

# ログ行をパースする関数
def parse_log_line(line):
    # 一般的なDLTログの書式を想定した正規表現
    # [時刻] LEVEL APID/CTID メッセージ
    match = re.match(r'^\[(.+)\]\s*([A-Z]{3})\s*([A-Z0-9]+)/([A-Z0-9]+)\s*(.*)$', line)

    if match:
        timestamp, level, apid, ctid, message = match.groups()
        return DltLogEntry(timestamp.strip(), level.strip(), apid.strip(), ctid.strip(), message.strip(), line)
    return None

# ZIPファイルを読み込み、ログをパースする関数
def load_and_parse_logs(zip_filepath):
    parsed_logs = []
    log_file_found = False

    try:
        with zipfile.ZipFile(zip_filepath, 'r') as zf:
            for name in zf.namelist():
                # テキストファイル（.log, .txtなど）を探す
                if name.lower().endswith(('.log', '.txt')):
                    print(f"✅ ログファイル '{name}' を読み込み中...")
                    log_file_found = True
                    with zf.open(name) as log_file:
                        # バイナリファイルをテキストとして読み込む
                        for line in log_file.read().decode('utf-8', errors='ignore').splitlines():
                            entry = parse_log_line(line.strip())
                            if entry:
                                parsed_logs.append(entry)
                    break # 最初のログファイルのみを処理

            if not log_file_found:
                print("❌ ZIPファイル内にDLTログ形式のファイル (.log, .txt) が見つかりませんでした。")
                return None

    except FileNotFoundError:
        print(f"❌ ファイルが見つかりません: {zip_filepath}")
        return None
    except zipfile.BadZipFile:
        print(f"❌ 無効なZIPファイルです: {zip_filepath}")
        return None
    except Exception as e:
        print(f"❌ ログの読み込み中にエラーが発生しました: {e}")
        return None

    if not parsed_logs:
        print("⚠️ ログをパースできませんでした。ログの形式が想定と異なる可能性があります。")

    # 時系列順にソート（既にパース時に時系列順になっている前提だが、念のため）
    # DLTログではタイムスタンプによるソートが重要
    # parsed_logs.sort(key=lambda x: x.timestamp)

    return parsed_logs

# ログをフィルタリングする関数
def filter_logs(logs, apid=None, ctid=None, level=None, keyword=None):
    filtered = logs

    # 1. APIDフィルタリング
    if apid:
        filtered = [log for log in filtered if log.apid == apid.upper()]

    # 2. CTIDフィルタリング
    if ctid:
        filtered = [log for log in filtered if log.ctid == ctid.upper()]

    # 3. レベルフィルタリング（指定レベル以上のログを表示）
    if level:
        # トレースレベルの優先順位（高い順）
        level_map = {'FAT': 6, 'ERR': 5, 'WRN': 4, 'INF': 3, 'DEB': 2, 'VER': 1}
        target_level = level.upper()
        target_priority = level_map.get(target_level, 0)

        filtered = [log for log in filtered if level_map.get(log.level, 0) >= target_priority]

    # 4. キーワード検索
    if keyword:
        # 大文字・小文字を区別しない検索
        keyword_lower = keyword.lower()
        filtered = [log for log in filtered if keyword_lower in log.message.lower()]

    return filtered

# メインの対話型インターフェース
def interactive_analyzer(logs):
    if not logs:
        print("\nパースされたログがないため、分析を開始できません。")
        return

    print("\n" + "="*50)
    print("DLTログ解析シミュレーターへようこそ！")
    print("フィルター条件を入力して、ログを読み解く練習をしましょう。")
    print(f"総ログ行数: {len(logs)}")
    print("="*50 + "\n")

    while True:
        # ユーザー入力の取得
        print("\n--- フィルタリング条件 ---")
        apid_input = input("APID (例: APPA, 空欄で全件): ").strip()
        ctid_input = input("CTID (例: CTRL, 空欄で全件): ").strip()
        level_input = input("最小レベル (例: ERR, WARN, 空欄で全件): ").strip()
        keyword_input = input("キーワード検索 (例: timeout, 空欄で全件): ").strip()

        print("\n🔍 フィルタリング中...")

        # フィルタリングの実行
        filtered_logs = filter_logs(
            logs,
            apid=apid_input if apid_input else None,
            ctid=ctid_input if ctid_input else None,
            level=level_input if level_input else None,
            keyword=keyword_input if keyword_input else None
        )

        # 結果の表示
        print("\n" + "~"*50)
        print(f"結果: {len(filtered_logs)} 件のログが一致しました (時系列順)")
        print("~"*50)

        if filtered_logs:
            # 時系列順に読む練習のため、各ログを表示
            for i, log in enumerate(filtered_logs):
                # 動作の流れを追う練習を支援するため、行番号を表示
                print(f"[{i+1:03d}] {log}")
        else:
            print("一致するログ行はありませんでした。条件を変えて試してみましょう。")

        # 継続確認
        print("\n" + "="*50)
        command = input("続行しますか？ (y/n, または 'q'で終了): ").lower()
        if command in ('n', 'q'):
            print("DLTログ解析の練習お疲れ様でした！またのご利用をお待ちしております。👋")
            break

if __name__ == "__main__":
    # ZIPファイルパスの指定
    # 実行前に dlt_logs.zip を作成し、カレントディレクトリに配置してください
    ZIP_FILE_PATH = "sample_dlt_drv.zip"

    # 1. ログのロードとパース
    all_logs = load_and_parse_logs(ZIP_FILE_PATH)

    # 2. 対話型解析の開始
    if all_logs:
        interactive_analyzer(all_logs)

✅ ログファイル 'sample_dlt_drv.log' を読み込み中...

DLTログ解析シミュレーターへようこそ！
フィルター条件を入力して、ログを読み解く練習をしましょう。
総ログ行数: 19


--- フィルタリング条件 ---
APID (例: APPA, 空欄で全件): COMM
CTID (例: CTRL, 空欄で全件): COMM
最小レベル (例: ERR, WARN, 空欄で全件): WRN
キーワード検索 (例: timeout, 空欄で全件): 

🔍 フィルタリング中...

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
結果: 0 件のログが一致しました (時系列順)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
一致するログ行はありませんでした。条件を変えて試してみましょう。

続行しますか？ (y/n, または 'q'で終了): y

--- フィルタリング条件 ---
APID (例: APPA, 空欄で全件): DRV
CTID (例: CTRL, 空欄で全件): COMM
最小レベル (例: ERR, WARN, 空欄で全件): WRN
キーワード検索 (例: timeout, 空欄で全件): 

🔍 フィルタリング中...

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
結果: 2 件のログが一致しました (時系列順)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[001] [2025-10-26 15:00:07.510] WRN DRV/COMM: Network connection unstable. Upload speed degraded for file_002.mp4.
[002] [2025-10-26 15:00:08.550] ERR DRV/COMM: Fatal server response on upload attempt for file_003.mp4. Retrying in 0.5s.

続行しますか？ (y/n, または 'q'で終了): y

--- フ

KeyboardInterrupt: Interrupted by user

# 新しいセクション